# Requesting External Data

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from pprint import pprint
import requests
import numpy as np
import pandas as pd
import time
from tqdm.notebook import trange, tqdm
import pdb

In [3]:
from request_discos import *
from request_uber_ares import *

# DiscosWeb

## Objects

In [5]:
operatorName = 'European Space Agency'
opCond = f"eq(operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(filter=filter)

objectIndex = [d['id'] for d in dataRequest]
objectAttributes = [d['attributes'] for d in dataRequest]
object_df = pd.DataFrame(objectAttributes,index=objectIndex)

object_df

,objectClass,xSectMin,height,xSectMax,span,xSectAvg,satno,diameter,depth,mass,vimpelId,shape,width,cosparId,name
8060,Payload,1.539380,1.20,2.278616,1.40,2.089159,8062,1.40,NaN,272.55,NaN,Cyl,NaN,1975-072A,COS B
9929,Payload,2.010619,1.30,10.138289,40.00,5.977604,9931,1.60,NaN,270.56,NaN,Cyl + 2 Ant,NaN,1977-029A,ESA GEOS 1
10421,Payload,1.266769,1.14,1.923754,1.27,1.770484,10423,1.27,NaN,164.52,NaN,Cyl,NaN,1977-102B,ISEE 2
10635,Payload,1.385929,4.20,9.289332,4.30,6.293122,10637,1.40,NaN,457.88,NaN,Oct Cyl + 2 Pan,NaN,1978-012A,IUE
10853,Payload,3.325538,1.60,5.361194,2.40,4.750615,10855,2.40,NaN,440.04,NaN,Hex Cyl,NaN,1978-044A,OTS 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66826,Payload Fragmentation Debris,NaN,NaN,NaN,NaN,NaN,48172,NaN,NaN,NaN,NaN,None,NaN,1991-050U,ERS-1 debris
66929,Rocket Fragmentation Debris,NaN,NaN,NaN,NaN,NaN,48260,NaN,NaN,NaN,NaN,None,NaN,1979-104AL,Ariane 1 fragmentation debris
66965,Payload Mission Related Object,NaN,NaN,NaN,NaN,NaN,48343,NaN,NaN,NaN,NaN,None,NaN,1981-057G,CAT 3 operational debris
67858,Payload,0.010000,0.20,0.030000,0.20,0.025000,49068,NaN,0.1,2.00,NaN,Box,0.1,2021-073C,Sunstorm


## Fragmentations

In [6]:
operatorName = 'European Space Agency'
opCond = f"eq(objects.operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(pageName="/api/fragmentations",filter=filter,sort='epoch',include='objects')

fragIndex = [d['id'] for d in dataRequest]
fragAttributes = [d['attributes'] for d in dataRequest]
frag_df = pd.DataFrame(fragAttributes,index=fragIndex)
frag_df['ObjectID'] = [d['relationships']['objects']['data'][0]['id'] for d in dataRequest]

frag_df

,latitude,comment,epoch,longitude,altitude,eventType,ObjectID
115,None,This mission was the inaugural flight of the A...,1980-03-31,None,None,Ariane Upper Stage,11656
467,None,Provisional breakup,1984-02-27,None,None,Ariane Upper Stage,14420
353,None,Provisional breakup,1984-07-04,None,None,Ariane Upper Stage,14784
427,None,,1987-06-07,None,None,Ariane Upper Stage,15873
11,None,Above elements are initial published values fo...,1987-09-19,None,None,Ariane Upper Stage,18349
337,None,ERS-2 anomaly,2003-07-24,None,None,ERS/SPOT Class,23557
695,None,A single fragment was catalogued. The breakup ...,2007-04-15,None,None,ERS/SPOT Class,23557
748,None,"On 22 May 2007 at about 0050 UTC, the spin-sta...",2007-05-22,None,None,Small Impactor,27506
697,None,Three fragments with consecutive satellite cat...,2009-04-25,None,None,ERS/SPOT Class,23557
324,None,ERS-1 anomalous event,2009-06-03,None,None,ERS/SPOT Class,21571


## Launches

In [7]:
operatorName = 'European Space Agency'
opCond = f"eq(objects.operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(pageName="/api/launches",filter=filter,sort='epoch',include='objects,site')

launchIndex = [d['id'] for d in dataRequest]
launchAttributes = [d['attributes'] for d in dataRequest]
launch_df = pd.DataFrame(launchAttributes,index=launchIndex)
launch_df['SiteID'] = [d['relationships']['site']['data']['id'] for d in dataRequest]

launchObjects = []
for d in dataRequest:
    objectData = d['relationships']['objects']['data']
    launchObjects.append([obj['id'] for obj in objectData])
launch_df['ObjectIDs'] = launchObjects

launch_df.head(10)

,failure,epoch,cosparLaunchNo,flightNo,SiteID,ObjectIDs
1962,False,1975-08-09T01:48:00+00:00,1975-072,602/D113,19,"[52335, 8061, 8060]"
245,False,1977-04-20T10:15:00+00:00,1977-029,617/D130,10,"[9932, 9931, 9930, 9929]"
1603,False,1977-10-22T13:53:00+00:00,1977-102,623/D135,10,"[10423, 10422, 10421, 10420]"
1605,False,1977-11-23T01:35:00+00:00,1977-108,620/D136,10,"[48118, 48117, 13904, 10948, 10488, 10487]"
2489,False,1978-01-26T17:36:00+00:00,1978-012,628/D138,10,"[37759, 32997, 10721, 10636, 10635]"
126,False,1978-05-11T22:59:00+00:00,1978-044,627/D141,10,"[10855, 10854, 10853]"
991,False,1978-07-14T10:43:00+00:00,1978-071,631/D143,10,"[10981, 10980, 10979]"
1357,False,1979-12-24T17:14:38+00:00,1979-104,L01,23,"[66929, 40776, 40779, 40778, 40777, 40775, 407..."
361,False,1981-06-19T12:32:59+00:00,1981-057,L03,23,"[66965, 48120, 48119, 20834, 14122, 12559, 125..."
2654,False,1981-12-20T01:29:00+00:00,1981-122,L04,23,"[13008, 13007, 13022]"


## Launch Sites

In [8]:
operatorName = 'European Space Agency'
opCond = f"eq(launches.objects.operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(pageName="/api/launch-sites",filter=filter,include='operators')

launchsiteIndex = [d['id'] for d in dataRequest]
launchsiteAttributes = [d['attributes'] for d in dataRequest]
launchsite_df = pd.DataFrame(launchsiteAttributes,index=launchsiteIndex)

launchsite_df

,pads,latitude,constraints,longitude,altitude,azimuths,name
7,None,13.900000,None,80.400000,0.0,"[{'lower': 102.0, 'display': '[102,139]', 'low...",Sriharikota Space Center
10,[LC46],28.500000,None,-81.000000,0.0,"[{'lower': 37.0, 'display': '[37,120]', 'lower...",Kennedy Space Center (ETR)
19,"[ABRESA2, ABRESB3, BMRSA1, RW30/12PAWA, 576E, ...",34.700000,None,-120.600000,0.0,"[{'lower': 147.0, 'display': '[147,201]', 'low...",Vandenberg AFB (WTR)
21,"[LA2?, LA1, LA2]",41.300000,None,100.300000,0.0,"[{'lower': 135.0, 'display': '[135,153]', 'low...",Jiuquan SLC (Shuang Cheng Tzu)
23,"[ELA1, ELA2, Europa, ELA3, Diamant]",5.200000,None,-52.800000,0.0,"[{'lower': -10.5, 'display': '[-10.5,93.5]', '...",Guiana Space Center (Kourou)
28,"[LC81, LC1, LC175/59, LC1/LC31?, LC45P/2, LC20...",45.600000,None,63.400000,90.0,"[{'lower': 35.1, 'display': '[35.1,35.1]', 'lo...",Baikonur Cosmodrome (Tyuratam)
29,"[LC32/2?, LC16-2, LC132/1, LC32/2, LC43-4?, LC...",62.800000,None,40.600000,0.0,"[{'lower': -13.0, 'display': '[-13,62]', 'lowe...",Plesetsk Cosmodrome


## Launch Vehicles

In [11]:
operatorName = 'European Space Agency'
opCond = f"eq(launches.objects.operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(pageName="/api/launch-vehicles",filter=filter,include='launches')

vehicleIndex = [d['id'] for d in dataRequest]
vehicleAttributes = [d['attributes'] for d in dataRequest]
vehicle_df = pd.DataFrame(vehicleAttributes,index=vehicleIndex)b

vehicleLaunches = []
for d in dataRequest:
    launchData = d['relationships']['launches']['data']
    vehicleLaunches.append([launch['id'] for launch in launchData])
vehicle_df['LaunchIDs'] = vehicleLaunches

vehicle_df.head(3)

,successfulLaunches,leoCapacity,height,span,diameter,failedLaunches,depth,mass,width,name,ssoCapacity,thrustLevel,escCapacity,geoCapacity,gtoCapacity,numStages,LaunchIDs
6,1,4900.0,None,None,4.0,0,None,240000.0,None,Ariane 40 H10+,NaN,752000.0,NaN,NaN,2050.0,3,[331]
7,97,23000.0,None,None,7.4,3,None,705000.0,None,Proton-M/Briz-M,NaN,1670000.0,NaN,3700.0,6150.0,4,"[6193, 6717, 5012, 5144, 4816, 5693, 5721, 634..."
25,9,4500.0,None,None,3.8,2,None,210000.0,None,Ariane 1,NaN,69300.0,730.0,NaN,1835.0,3,"[361, 806, 1357, 1405, 1452, 1745, 2654, 3216,..."


# Uberlog and ARES

In [7]:
resp_json = requestUberlog('gaia')
resp_json

{'count': 1691,
 'entries': [{'author': 'admin',
   'creationDate': 1518427647133,
   'eventDate': 1518427646000,
   'group': '',
   'id': '5a815dffd242d32bffd2da21',
   'mission': 'gaia',
   'severity': 'Error',
   'text': '<div>Next 2 passes not covered.</div>'}],
 'limit': 10,
 'skip': 0}

In [30]:
mission = 'gaia'
parameter = 'x1234'

t1 = time.strptime("01 Oct 2018 00:00", "%d %b %Y %H:%M")
t2 = time.strptime("02 Oct 2018 10:00", "%d %b %Y %H:%M")
intervalStart = int(time.mktime(t1)*1000.)
intervalEnd = int(time.mktime(t2)*1000.)

resp_json = requestAres(mission, parameter, intervalStart, intervalEnd, pageName="data", operation=None)
resp_json

[{'data': [{'timestamp': 1538300000000, 'value': 2.656369948440532},
   {'timestamp': 1538350000000, 'value': 1.0482780028455796},
   {'timestamp': 1538400000000, 'value': 0.5311632146975165}],
  'mission': 'gaia',
  'parameter': 'x1234'}]